# Bitfinex Analysis BuyLowSellHigh

## Introduction

In this notebook, we will analyze the profitability of the BLOSEH strategy.

## BLOSEH

The Buy Low Sell High follows a simple principle:

1. Checkk all pairs. Find the pair(s) with the highest drop in price.
2. Buy some quantity of the pairs.
3. Hold the pair until the price goes up by X %, where X considers the desired profit + transaction fees.

In [5]:
import pandas as pd
from matplotlib import pyplot as plt
import os
from datetime import datetime as dtt, timedelta
import numpy as np
import warnings
from multiprocessing import Pool
import workers

warnings.filterwarnings('ignore')

## Data

We will only consider pairs connected to USD, to simplify the analysis. Similarly, To avoid missing data, we will only consider data from one year in the past.

In [6]:
files = [f for f in os.listdir('../../gatherers/bitfinex/data/') if 'USD.csv' in f]

start_date = dtt.now()
end_date = dtt.now() - timedelta(days=365)
pairs_dfs = {}

for f in files:
    if "TESTUSD" in f:
        continue
    df = pd.read_csv(f'../../gatherers/bitfinex/data/{f}', sep=";", index_col='mts')
    df = df[df.index > end_date.timestamp()*1000]
    pairs_dfs[f.replace('.csv','')] = df

In [8]:
STEP_SIZE = 1 # hours
MIN_PROFIT = 0.05 # 0.1 = 10% price increase to sell

analysis_start = end_date + timedelta(hours=STEP_SIZE)
analysis_end = start_date

current_batch_start = analysis_start - timedelta(hours=STEP_SIZE)
current_batch_end = analysis_start

dinero = 500

max_holdings = 30

current_holdings = pd.DataFrame(columns=['id','mts', 'pair', 'amount', 'price'])
current_holdings = current_holdings.set_index('id')
historical_transactions = pd.DataFrame(columns=['id', 'mts', 'pair', 'amount', 'price', 'type'])
historical_transactions = historical_transactions.set_index('id')

MONTHLY_INVESTMENT = 300

transaction_id_counter = 0

buy_more_this_iteration = False

current_month = analysis_start.month

while current_batch_end < analysis_end:
    # sanity check
    if current_holdings.shape[0] > max_holdings:
        print("WOOPS! More holdings than allowed")
        exit()

    # LOCAL DATA GENERATION
    local_chunks = {}
    for pair in pairs_dfs.keys():
        current_pair_df = pairs_dfs[pair]
        local_chunk = current_pair_df[(current_pair_df.index > current_batch_start.timestamp()*1000) & (current_pair_df.index < current_batch_end.timestamp()*1000)]
        if local_chunk.empty:
            continue
        local_chunk = local_chunk.sort_values(by='mts', ascending=False) # earliest first
        local_chunk['avg'] = (local_chunk['open'] + local_chunk['low'] + local_chunk['high'] + local_chunk['close']) / 4
        local_chunks[pair] = local_chunk

    # SELLING 
    sold_this_chunk = []
    for i, row in current_holdings.iterrows():
        pair = row['pair']
        try:
            local_chunk = local_chunks[pair]
        except KeyError as e: # no information about the current holding pair in this chunk
            continue
        if row['price'] * (1 + MIN_PROFIT) < local_chunk['high'].iloc[0]: # sell
            amount_to_sell = row['amount']
            sell_price = row['price'] * (1 + MIN_PROFIT) * row['amount']
            sell_price_with_fee = sell_price * (1 - 0.002)
            print(f"Sold {amount_to_sell} of {pair} for {sell_price_with_fee} on {current_batch_end}")
            print(f"Made profit of {row['amount'] * row['price'] * MIN_PROFIT}")
            sold_this_chunk.append(i)
            historical_transactions = historical_transactions.append({
                'id': i,
                'mts': current_batch_end,
                'pair': pair,
                'amount': row['amount'],
                'price': sell_price_with_fee, 
                'type': 'SELL'
            }, ignore_index=True)
            dinero += sell_price_with_fee
    current_holdings = current_holdings.drop(index=sold_this_chunk)


    # CHECK CURRENT HOLDINGS
    if current_holdings.shape[0] < max_holdings:
        buy_more_this_iteration = True

    if buy_more_this_iteration:
        # FIND LOWEST
        slopes = []
        for pair in local_chunks.keys():
            local_chunk = local_chunks[pair]
            slope = np.polyfit(local_chunk.index, local_chunk['avg'], 1)[0]
            slopes.append((slope, pair))
        slopes = sorted(slopes, key=lambda tup: tup[0]) # sort by slopes lowest to highest
        to_buy = slopes[:(max_holdings - current_holdings.shape[0])] # ensures we never buy more than max_holdings
        for slope, pair in to_buy:
            local_chunk = local_chunks[pair]
            buy_price = local_chunk['avg'].iloc[0]
            dinero_to_spend = dinero / (max_holdings - current_holdings.shape[0])
            dinero -= dinero_to_spend
            amount_to_buy = (dinero_to_spend / buy_price) * (1-0.001) # maker fee
            print(f"Bought {amount_to_buy} of {pair} for {buy_price * amount_to_buy} on {current_batch_end}")
            print("---------------------------------------")
            current_holdings = current_holdings.append({
                'id': transaction_id_counter,
                'mts': current_batch_end,
                'pair': pair,
                'amount': amount_to_buy,
                'price': buy_price, 
            }, ignore_index=True)
            historical_transactions = historical_transactions.append({
                'id': transaction_id_counter,
                'mts': current_batch_end,
                'pair': pair,
                'amount': amount_to_buy,
                'price': buy_price, 
                'type': 'BUY'
            }, ignore_index=True)
            transaction_id_counter += 1   
    else:
        pass

    # MONTHLY INVESTMENT
    if current_batch_start.month != current_month:
        dinero += MONTHLY_INVESTMENT
        current_month = current_batch_start.month

    current_value_of_assets = sum(current_holdings['amount'] * current_holdings['price'])
    print(f"CURRENT MONEY: {current_value_of_assets + dinero}")
    current_batch_start = current_batch_end
    current_batch_end = current_batch_end + timedelta(hours=STEP_SIZE)

print(f"FINAL DINERO: {dinero}")

Bought 0.12462574850299402 of tCOMPUSD for 16.65 on 2020-12-27 19:36:34.206487
---------------------------------------
Bought 0.7349857637892597 of tFILUSD for 16.649999999999995 on 2020-12-27 19:36:34.206487
---------------------------------------
Bought 3.7920628594203714 of tHEZUSD for 16.65 on 2020-12-27 19:36:34.206487
---------------------------------------
Bought 1.1467731937461254 of tBCHABCUSD for 16.649999999999995 on 2020-12-27 19:36:34.206487
---------------------------------------
Bought 0.6056527590847913 of tMLNUSD for 16.65 on 2020-12-27 19:36:34.206487
---------------------------------------
Bought 1.3428502298572462 of tLINKUSD for 16.65 on 2020-12-27 19:36:34.206487
---------------------------------------
Bought 2.5230904448367557 of tSUNUSD for 16.64999999999999 on 2020-12-27 19:36:34.206487
---------------------------------------
Bought 3.1942599245080303 of tDOTUSD for 16.649999999999995 on 2020-12-27 19:36:34.206487
---------------------------------------
Bought 

KeyboardInterrupt: 

In [ ]:
 1000 * 1.05 = 1050